# Analyze the Treasure Archive

In [ ]:
from ipywidgets import interact
import ipywidgets as widgets
from ipywidgets import Layout
import webbrowser    # for opening web pages
import networkx as nx
import networkx.algorithms.community as nxcom
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
%matplotlib inline
from IPython.display import display, clear_output
import pickle
import numpy as np

In [ ]:
with open('pickle/treasury.p', 'rb') as f: 
    G = pickle.load(f)

# Compute some node centrality measures
We may compute a variety of characteristics of each node in our graph. There are different ways to conceptualize the centrality  of a node in a graph. The number of connections of a node to other nodes in the graph contributes to its *degree centrality*. *In degree centrality* refers to the number of incoming connections (where the node is target), whereas *out degree centrality* relates to the number of outgoing connections (where the node source). *Betweenness centrality* is based on the number of shortest paths between any two nodes in the graph that go through this node. *Eigenvector centrality* is a measure that takes into account the centrality of the nodes that the node under consideration connects to. A node that connects to many low-ranking nodes will receive a low eigenvector centrality score. A node that connects to a few well-connected nodes will receive a high eigenvector centrality score.

*Hub* and *authority*, finally, are interrelated measures that were originally developed for ranking web pages. A node with high hub value points towards multiple nodes with high authority value. A node with high authority value has incoming connections from multiple nodes with high hub  value. Hub and authority are thus defined by each other and the process of computing hub and authority values is an iterative one. If `nx.hits()` raises a "PowerIterationFailedConvergence" error, you may increase the maximum number of iterations. The default value is `max_iter=100`; use, for instance `hub, authority = nx.hits(G, max_iter = 1000)`.

Each of these computations results in a dictionary in which the nodes are keys.

In [ ]:
eigenvector = nx.eigenvector_centrality(G)
degree_centrality = nx.degree_centrality(G)
in_degree_centrality = nx.in_degree_centrality(G)
out_degree_centrality = nx.out_degree_centrality(G)
closeness_centrality = nx.closeness_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G)
hub, authority = nx.hits(G, max_iter=600)

Print the first 5 highest scoring nodes for each of the centrality measure. In each of these measures we see more or less the same names popping up, but not in the same order. 

In [ ]:
centrality = [eigenvector, degree_centrality, in_degree_centrality, out_degree_centrality, closeness_centrality, betweenness_centrality, hub, authority]
centrality_labels = ["eigenvector", "degree centrality", "in degree centrality",
                     "out degree centrality", "closeness centrality",
                     "betweenness centrality", "hub", "authority"]
for idx, m in enumerate(centrality):
    s = sorted(m, key=m.get, reverse=True)[:5]
    print(f"{centrality_labels[idx]}: {s}")

# Cliques
Cliques are sets of nodes that are all connected to each other. That is, if we have four nodes, A, B, C, and D, all possible connections exist: 
> A --- B</br>
> A --- C</br>
> A --- D</br>
> B --- C</br>
> B --- D</br>
> C --- D</br>

Cliques may represent powerful centers in a graph, with  information flowing freely within each clique. Cliques are defined only for undirected graphs, so we first have to transform our graph. In order to compute cliques we need to decide the minimum number of nodes - in this case 4.
See https://orbifold.net/default/community-detection-using-networkx/

A related concept is the k-clique community. A k-clique community consists of adjacent cliques of at least *k* members. Adjacent cliques share at least *k-1* nodes. As we will see, k-clique communities tend to overlap. That is, there are powerful actors who belong to multiple such communities and connect them to each other.

The following code needs more documentation. Currently, the three clique-communities are colored yellow, blue and brown, but it is necessary to give those actors who appear in more than one clique-community a different color (right now they are probably colored as belonging to the last clique that come around in the loop). Coloring of internal vs. external edges currently does not work.

The output of `k_clique_communities()` is a list of frozen sets. The clique communities are sorted from largest to smallest.

In [ ]:
H = nx.DiGraph.to_undirected(G)
k_clique_com = sorted(nxcom.k_clique_communities(H, 4), key=len, reverse=True)
# Count the communities
N = len(k_clique_com)
print(f"The graph has {N} clique communities.")

Each node in the graph receives an attribute called `cliques`. The default is `{0}` (a set). In iterating through the nodes in the clique communities, the attribute `cliques` is updated so that all nodes that belong to the first k-clique community now have the attribute `cliques = {1}` (again, a set), etc. Nodes that belong to more than one clique community will have more than one clique number in the `cliques` attribute. Each node also receives a `color` attribute, an integer which defaults to `0` for nodes that do not belong to a clique. Nodes that do belong to a clique receive the number of their clique in the `color` attribute, with the exception of the nodes that belong to multiple cliques. Those nodes are important, because they represent the people who hold the various clique communities together (or communicate between them). Those nodes, therefore, receive a separate value that equals the number of clique communities plus 1.

In [ ]:
nx.set_node_attributes(H, {0}, "cliques") # set {0} as default
nx.set_node_attributes(H, 0, "color")     # set 0 as default
for c, v_c in enumerate(k_clique_com):
    for v in v_c:
        if H.nodes[v]['color'] > 0:  # node is already in a clique
            H.nodes[v]['cliques'].add(c+1) # add clique no. to the set
            H.nodes[v]['color'] = N + 1 # special value for nodes in multiple cliques
        else:
            H.nodes[v]['cliques'] = {c+1}
            H.nodes[v]['color'] = c + 1
        # Add 1 to save 0 for nodes that are not in a clique
            

Set edge attributes with the same method. The edge attribute `clique` defaults to zero. Edges between two nodes that belong to the same clique (internal edges) receive the value of that clique. This is done by finding the intersection of the `cliques` attributes of the the nodes. If that intersection is empty (the two nodes do not share membership of a clique community) the value remains 0. If the `cliques` attributes of two nodes do intersect (that is, they share membership of at least one k-clique community), the value of the edge attribute becomes the lowest value in the intersection.

In [ ]:
nx.set_edge_attributes(H, 0, "clique")  # default is 0
for v, w, in H.edges:
    c = H.nodes[v]['cliques'].intersection(H.nodes[w]['cliques']) # do the two nodes share a community?
    if c:
        # Internal edge, mark with community
        H.edges[v, w]['clique'] = min(c)

Select nodes to display. Nodes that belong to a k-clique community and nodes that have a degree greater than 8 are admitted.

In [ ]:
nodes = [x for x, y in H.nodes(data=True) if y["color"] > 0 or y["degree"] > 8] # select nodes
I = H.subgraph(nodes)

For the color scheme we use a function for creating a discrete colormap, made available by Jake VanderPlas on his github account. The [tool](https://gist.github.com/jakevdp/91077b0cae40f8f8244a) is slightly adapted for the present purposes.

In [ ]:
def discrete_cmap(N, base_cmap=None):
    """Create an N-bin discrete colormap from the specified input map
    https://gist.github.com/jakevdp/91077b0cae40f8f8244a
    by Jake VanderPlas"""
    base = plt.cm.get_cmap(base_cmap)
    color_list = base(np.linspace(0, 1, N))
    return color_list

In [ ]:
colmap = discrete_cmap(N+2, "jet")
colmap[0] = [0, 0, 0, 1]  # 0 = black

Use the resulting colormap to assign node colors and edge colors of our subgraph.

In [ ]:
node_color = [colmap[I.nodes[v]['color']] for v in I.nodes]
edge_color = [colmap[I.edges[v, w]['clique']] for v, w in I.edges]

In [ ]:
labels = nx.get_node_attributes(I, 'name')

In [ ]:
label_pos = {}
pos = nx.spring_layout(I)
for k, v in pos.items():   # shift position of label slightly away from the node
    label_pos[k] = (v[0], v[1]+0.03)
fig = plt.figure(figsize=(15, 10))
nx.draw(
        I,
        pos=pos,
        node_color=node_color,
        edge_color=edge_color, 
        #node_size = node_size, 
        with_labels = False,
        cmap = colmap, 
        width = 3, 
        alpha = .65
        #cmap= plt.cm.jet
        )
nx.draw_networkx_labels(I, label_pos, labels, font_size=16)
# the following are fake "lines" used to create the legend.
legend_elements = [Line2D([1], [0], marker='o', color='w', label=str(i+1),
                          markerfacecolor=colmap[i+1], markersize=15) for i, c in enumerate(k_clique_com)]
no_com = [Line2D([0], [0], marker='o', color='w', label="None",
                          markerfacecolor=colmap[0], markersize=15)]
mult_com = [Line2D([0], [0], marker='o', color='w', label="Multiple",
                          markerfacecolor=colmap[N+1], markersize=15)]
legend_elements = no_com + legend_elements + mult_com
fig.legend(handles=legend_elements)
plt.show()

In [ ]:
cliques = nx.find_cliques(H)
cliques4 = [clq for clq in cliques if len(clq) >= 4]
nodes = set(n for clq in cliques4 for n in clq)
print(len(cliques4))
h = H.subgraph(nodes)
fig = plt.figure(figsize=(15, 10))
nx.draw(h, with_labels=True, font_size = 24)
# from https://stackoverflow.com/questions/25222322/networkx-create-new-graph-of-all-nodes-that-are-a-part-of-4-node-clique

# Import Bokeh
The networkx module has facilities for displaying a network, but the options are limited. The Bokeh module has a much wider range of possibilities for drawing, inspecting, and exporting a network graph. The `from_networkx` function in Bokeh allows efficient import of nodes and edges.

In [ ]:
from bokeh.io import output_file, show
from bokeh.models import (BoxSelectTool, Circle, EdgesAndLinkedNodes, HoverTool,
                          MultiLine, NodesAndLinkedEdges, Plot, Range1d, TapTool,
                         BoxZoomTool, ResetTool, OpenURL, CustomJS, Column, SaveTool)
from bokeh.palettes import Spectral4
from bokeh.plotting import figure, output_notebook
from bokeh.models.graphs import from_networkx
from bokeh.models import TextInput, Button
output_notebook()

# Tooltips for Nodes
This graph provides tooltips for each node, containing the name and the degree. By computing other characteristics of the node, one may easily display other attributes in the tooltip. 

The edges are not displayed by default, but light up when the mouse hovers over a node.

In [ ]:
plot = Plot(plot_width=900, plot_height=900,
            x_range=Range1d(-1.1, 1.1), y_range=Range1d(-1.1, 1.1))
plot.title.text = "Treasure Archive Drehem"

node_hover_tool = HoverTool(tooltips=[("name", "@name"), ("degree", "@degree")])
plot.add_tools(node_hover_tool, BoxZoomTool(), ResetTool())

graph_renderer = from_networkx(G, nx.spring_layout, scale=1, center=(0, 0))

graph_renderer.node_renderer.glyph = Circle(size='node_size', fill_color=Spectral4[0])
graph_renderer.edge_renderer.glyph = MultiLine(line_color=Spectral4[1], line_alpha=0.8, line_width=3)
plot.renderers.append(graph_renderer)

output_file("interactive_graphs.html")
show(plot)

In [ ]:
plot = figure(plot_width=700, plot_height=700,
            x_range=Range1d(-1.1,1.1), y_range=Range1d(-1.1,1.1))

node_hover_tool = HoverTool(tooltips=[("name", "@name"), ("degree", "@degree")])
plot.add_tools(node_hover_tool)
plot.title.text = "Drehem Treasure Archive: the Nodes"

r = from_networkx(G, nx.circular_layout, scale=1, center=(0,0))

r.node_renderer.glyph = Circle(size='node_size', fill_color='#2b83ba')
r.node_renderer.hover_glyph = Circle(size='node_size', fill_color='#abdda4')

r.edge_renderer.glyph = MultiLine(line_alpha=0, line_width='weight')  # zero line alpha
r.edge_renderer.hover_glyph = MultiLine(line_color='#abdda4', line_width=5)

r.inspection_policy = NodesAndLinkedEdges()
plot.renderers.append(r)

show(plot)

In [ ]:
plot = Plot(plot_width=1000, plot_height=900,
            x_range=Range1d(-2, 2), y_range=Range1d(-2, 2))
plot.title.text = "Drehem Treasure Archive"

graph_renderer = from_networkx(G, nx.circular_layout, scale=1.9, center=(0, 0))
#graph_renderer.edge_renderer.data_source.data["line_width"] = [G.get_edge_data(a,b)['weight'] for a, b in G.edges()]
#graph_renderer.edge_renderer.glyph.line_width = {'field': 'line_width'}
graph_renderer.edge_renderer.glyph.line_alpha = 0.8
graph_renderer.node_renderer.selection_glyph = Circle(size='degree', fill_color=Spectral4[2])
graph_renderer.node_renderer.hover_glyph = Circle(size='degree', fill_color=Spectral4[1])
graph_renderer.node_renderer.glyph = Circle(size='degree', fill_color=Spectral4[0])
graph_renderer.edge_renderer.glyph = MultiLine(line_color="#CCCCCC", line_alpha=0.8, line_width = 'weight')
graph_renderer.edge_renderer.selection_glyph = MultiLine(line_color=Spectral4[2], line_width = 'weight')
graph_renderer.edge_renderer.hover_glyph = MultiLine(line_color=Spectral4[1], line_width =  'weight')

graph_renderer.inspection_policy = NodesAndLinkedEdges()
graph_renderer.selection_policy = EdgesAndLinkedNodes()

node_hover_tool = HoverTool(tooltips=[("name", "@name"), ('degree', '@value')])
plot.renderers.append(graph_renderer)
plot.add_tools(node_hover_tool, BoxZoomTool(), ResetTool())
output_file("vis/interactive_graphs.html")
show(plot)

In [ ]:
plot = Plot(plot_width=900, plot_height=900,
            x_range=Range1d(-1.1, 1.1), y_range=Range1d(-1.1, 1.1))
plot.title.text = "Treasure Archive Drehem"

node_hover_tool = HoverTool(tooltips=[("name", "@name"), ("degree", "@degree")])
plot.add_tools(node_hover_tool, BoxZoomTool(), ResetTool())

graph_renderer = from_networkx(G, nx.spring_layout, scale=1, center=(0, 0))

graph_renderer.node_renderer.glyph = Circle(size='node_size', fill_color=Spectral4[0])
graph_renderer.edge_renderer.glyph = MultiLine(line_color=Spectral4[1], line_alpha=0.8, line_width=3)
plot.renderers.append(graph_renderer)

#output_file("interactive_graphs.html")
# when using output_file() bokeh will open a new tab
# when using output_file() continue with reset_output()
show(plot)

In [ ]:
plot = Plot(plot_width = 900, plot_height = 900,
            x_range = Range1d(-1.1, 1.1), y_range = Range1d(-1.1, 1.1))
plot.title.text = "Drehem Treasure Archive"

plot.add_tools(HoverTool(tooltips = [('Text ID', '@id_text'), ('start', '@start'), ('end', '@end')]), 
               TapTool(), BoxSelectTool(), ResetTool(), BoxZoomTool(), SaveTool())

graph_renderer = from_networkx(G, nx.circular_layout, scale = 1, center = (0, 0))

graph_renderer.node_renderer.glyph = Circle(size = 'node_size', fill_color = Spectral4[0])
graph_renderer.node_renderer.selection_glyph = Circle(size = 'node_size', fill_color = Spectral4[2])
graph_renderer.node_renderer.hover_glyph = Circle(size = 'node_size', fill_color = Spectral4[1])

graph_renderer.edge_renderer.glyph = MultiLine(line_color = "#CCCCCC", line_alpha = 0.8, line_width = 5)
graph_renderer.edge_renderer.selection_glyph = MultiLine(line_color = Spectral4[2], line_width = 5)
graph_renderer.edge_renderer.hover_glyph = MultiLine(line_color = Spectral4[1], line_width = 5)

graph_renderer.selection_policy = NodesAndLinkedEdges()
graph_renderer.inspection_policy = EdgesAndLinkedNodes()

plot.renderers.append(graph_renderer)

info_text_ids = TextInput(title = 'Text IDs:', value = '')
info_start_end = TextInput(title = 'Start => End Values:', value = '')
url = "http://oracc.org/epsd2/admin/ur3/"
esource = graph_renderer.edge_renderer.data_source
code = """  start_end_values = []
            text_ids = []
            for(idx in esource.selected.indices)
            {
                index = esource.selected.indices[idx]
                start_end_values.push(esource.data['start'][index] + '=>' + esource.data['end'][index] ); 
                text_ids.push(esource.data['id_text'][index]) 
            }
            info_start_end.value = String(start_end_values);
            info_text_ids.value = String(text_ids);"""
code2 = """ urlnew = url.concat(info_text_ids.value);
            window.open(urlnew)"""
callback = CustomJS(args = dict(esource = esource, 
                                info_text_ids = info_text_ids, 
                                info_start_end = info_start_end), 
                    code = code)
plot.select_one(TapTool).callback = callback
button = Button(label="Click to open text editions", button_type="success")
button.js_on_click(CustomJS(args = dict(url=url, info_text_ids=info_text_ids), 
                        code=code2))
show(Column(plot, info_start_end, info_text_ids, button))

In [ ]:
plot = figure(plot_width=700, plot_height=700,
            x_range=Range1d(-1.1,1.1), y_range=Range1d(-1.1,1.1))

node_hover_tool = HoverTool(tooltips=[("name", "@name"), ("degree", "@degree")])
plot.add_tools(node_hover_tool)
plot.title.text = "Drehem Treasure Archive: the Nodes"

r = from_networkx(G, nx.circular_layout, scale=1, center=(0,0))

r.node_renderer.glyph = Circle(size='node_size', fill_color='#2b83ba')
r.node_renderer.hover_glyph = Circle(size='node_size', fill_color='#abdda4')

r.edge_renderer.glyph = MultiLine(line_alpha=0, line_width='weight')  # zero line alpha
r.edge_renderer.hover_glyph = MultiLine(line_color='#abdda4', line_width=5)

r.inspection_policy = NodesAndLinkedEdges()
plot.renderers.append(r)

show(plot)

In [ ]:
plot = Plot(plot_width=1000, plot_height=900,
            x_range=Range1d(-2, 2), y_range=Range1d(-2, 2))
plot.title.text = "Drehem Treasure Archive"

graph_renderer = from_networkx(G, nx.circular_layout, scale=1.9, center=(0, 0))
#graph_renderer.edge_renderer.data_source.data["line_width"] = [G.get_edge_data(a,b)['weight'] for a, b in G.edges()]
#graph_renderer.edge_renderer.glyph.line_width = {'field': 'line_width'}
graph_renderer.edge_renderer.glyph.line_alpha = 0.8
graph_renderer.node_renderer.selection_glyph = Circle(size='degree', fill_color=Spectral4[2])
graph_renderer.node_renderer.hover_glyph = Circle(size='degree', fill_color=Spectral4[1])
graph_renderer.node_renderer.glyph = Circle(size='degree', fill_color=Spectral4[0])
graph_renderer.edge_renderer.glyph = MultiLine(line_color="#CCCCCC", line_alpha=0.8, line_width = 'weight')
graph_renderer.edge_renderer.selection_glyph = MultiLine(line_color=Spectral4[2], line_width = 'weight')
graph_renderer.edge_renderer.hover_glyph = MultiLine(line_color=Spectral4[1], line_width =  'weight')

graph_renderer.inspection_policy = NodesAndLinkedEdges()
graph_renderer.selection_policy = EdgesAndLinkedNodes()

node_hover_tool = HoverTool(tooltips=[("name", "@name"), ('degree', '@value')])
plot.renderers.append(graph_renderer)
plot.add_tools(node_hover_tool, BoxZoomTool(), ResetTool())
# output_file("vis/interactive_graphs.html")
show(plot)

# Tooltips for Edges
The following code draws a network of the Drehem Treasure Archive. By selecting a node (click on the node) the node and all its direct neighbors are highlighted and the button below the drawing links to the collection of texts in ORACC from which these edges come.
# TODO
The list of P numbers is currently not unique - a P number may provide more than one relevant edge.

In [167]:
plot = Plot(plot_width = 900, plot_height = 900,
            x_range = Range1d(-1.1, 1.1), y_range = Range1d(-1.1, 1.1))
plot.title.text = "Drehem Treasure Archive"

plot.add_tools(HoverTool(tooltips = [('Text ID', '@id_text'), ('start', '@start'), ('end', '@end')]), 
               TapTool(), BoxSelectTool(), ResetTool(), BoxZoomTool(), SaveTool())

graph_renderer = from_networkx(G, nx.circular_layout, scale = 1, center = (0, 0))

graph_renderer.node_renderer.glyph = Circle(size = 'node_size', fill_color = Spectral4[0])
graph_renderer.node_renderer.selection_glyph = Circle(size = 'node_size', fill_color = Spectral4[2])
graph_renderer.node_renderer.hover_glyph = Circle(size = 'node_size', fill_color = Spectral4[1])

graph_renderer.edge_renderer.glyph = MultiLine(line_color = "#CCCCCC", line_alpha = 0.8, line_width = 5)
graph_renderer.edge_renderer.selection_glyph = MultiLine(line_color = Spectral4[2], line_width = 5)
graph_renderer.edge_renderer.hover_glyph = MultiLine(line_color = Spectral4[1], line_width = 5)

graph_renderer.selection_policy = NodesAndLinkedEdges()
graph_renderer.inspection_policy = EdgesAndLinkedNodes()

plot.renderers.append(graph_renderer)

info_text_ids = TextInput(title = 'Text IDs:', value = '')
info_start_end = TextInput(title = 'Start => End Values:', value = '')
url = "http://oracc.org/epsd2/admin/ur3/"
esource = graph_renderer.edge_renderer.data_source
code = """  start_end_values = []
            text_ids = []
            for(idx in esource.selected.indices)
            {
                index = esource.selected.indices[idx]
                start_end_values.push(esource.data['start'][index] + '=>' + esource.data['end'][index] ); 
                text_ids.push(esource.data['id_text'][index]) 
            }
            info_start_end.value = String(start_end_values);
            info_text_ids.value = String(text_ids);"""
code2 = """ urlnew = url.concat(info_text_ids.value);
            window.open(urlnew)"""
callback = CustomJS(args = dict(esource = esource, 
                                info_text_ids = info_text_ids, 
                                info_start_end = info_start_end), 
                    code = code)
plot.select_one(TapTool).callback = callback
button = Button(label="Click to open text editions", button_type="success")
button.js_on_click(CustomJS(args = dict(url=url, info_text_ids=info_text_ids), 
                        code=code2))
show(Column(plot, info_start_end, info_text_ids, button))